Función general de FISTA

In [ ]:
% Función general de FISTA
function [stats, x, var] = general_fista(opt, var, b, x, L)

    % Cálculo de la primera estadística previo a computar el algoritmo
    Ax = Ax_fun(x, opt, var);
    z = Ax - b;
    stats.fx(1) = 0.5 * sum(z(:).*z(:));  % Función de Error
    stats.l1(1) = norm(x.val .* var.c_mask, 1);  % Función de la norma l1
    stats.F(1) = stats.fx(1) + opt.lambda * stats.l1(1);  % Función de Costo

    % Inicio de conteo de tiempo
    var.time(1) = 0;
    tic
    y = x;
    t = 1;

    % Creación del vector de lambdas para Warm Start
    if opt.warms
        var.lambda_vec_init = var.lambda_vec;
        var.lambda_vec_ws = opt.ws_inc * var.lambda_vec;
    end
    maskS = ones(length(x.val), 1);

    % Reducción de tamaño por Screening
    if opt.screening
        var.mask_SC = screening_fun(b, opt, var, y);
        x.val = x.val .* var.mask_SC;
        y.val = y.val .* var.mask_SC;
    end

    % Primer paso de Cauchy
    i = 0;
    ATz = ATx_fun(z, opt, var);
    g = ATz;

    var.g(1, :) = g.val;
    var.alpha(1) = step_fun(y, g, opt, var, i);
    alphaCi = var.alpha(1);

    % Inicio del algoritmo iterativo
    for i = 1:L

        % Warm Start
        if opt.warms
            var.lambda_vec = warms_fun(opt, var, i);
        end

        % Cálculo de tamaño de paso de Cauchy de iter. previa para Yuan B
        if opt.stepT == 'YnB'
            ATz = ATx_fun(z, opt, var);
            g = ATz;
            alphaCi = 0;
            alphaCi = step_fun(z, y, g, opt, var, i, alphaCi);
        end

        % Guardar valores anteriores
        var.x_ant = x.val;
        var.g_ant = g.val;
        var.y_ant = y.val;

        % Shrinkage
        arg_p = y.val - var.alpha(i) .* g.val;
        v = abs(arg_p);
        mask = v > var.alpha(i) .* var.lambda_vec;
        mask = mask .* maskS;
        x.val = sign(arg_p) .* (mask.*v - var.alpha(i) .* var.lambda_vec);

        % Actualización de valores
        t_next = (1 + (1 + 4*t^2)^(1/2)) / 2;
        y.val = x.val + ((t - 1) / t_next) .* (x.val - var.x_ant);
        t = t_next;

        % Gradiente
        Ax = Ax_fun(y, opt, var);
        z = Ax - b;
        ATz = ATx_fun(z, opt, var);
        g = ATz;
        var.g(i+1, :) = g.val;

        % Tamaño de paso BB1
        var.alpha(i+1) = step_fun(y, g, opt, var, i, alphaCi);

        % Cálculo de estadísticas
        stats.fx(i+1) = 0.5*sum(z(:).*z(:));  % Función de Error
        stats.l1(i+1) = norm(y.val .* var.c_mask, 1);  % Función de la norma l1
        stats.F(i+1) = stats.fx(i+1) + opt.lambda .* stats.l1(i+1);  % F. de Costo
        maskS = ones(length(y.val), 1);
        var.time(i+1) = toc;
    end

end

### Función general de ISTA

In [ ]:
% Función general de ISTA
function [stats, x, var] = general_ista(opt, var, b, x, L)

    % Cálculo de la primera estadística previo a computar el algoritmo
    Ax = Ax_fun(x, opt, var);
    z = Ax - b;
    stats.fx(1) = 0.5 * sum(z(:).*z(:));  % Función de Error
    stats.l1(1) = norm(x.val .* var.c_mask, 1);  % Función de la norma l1 (tamaño)
    stats.F(1) = stats.fx(1) + opt.lambda.* stats.l1(1);  % Función de Costo

    % Inicio de conteo de tiempo
    var.time(1) = 0;
    tic

    % Creación del vector de lambdas para Warm Start
    if opt.warms
        var.lambda_vec_init = var.lambda_vec;
        var.lambda_vec_ws = opt.ws_inc * var.lambda_vec;
    end
    maskS = ones(length(x.val), 1);

    % Reducción de tamaño por Screening
    if opt.screening
        var.mask_SC = screening_fun(b, opt, var, x);
        x.val = x.val .* var.mask_SC;
    end

    % Primer paso de Cauchy
    i = 0;
    ATz = ATx_fun(z, opt, var);
    g = ATz;
    var.alpha = step_fun(x, g, opt, var, i);

    % Inicio del algoritmo iterativo
    for i = 1:L

        % Warm Start
        if opt.warms
            var.lambda_vec = warms_fun(opt, var, i);
        end

        % Guardar valores anteriores
        var.x_ant = x.val;
        var.g_ant = g.val;

        % Shrinkage
        arg_p = x.val - var.alpha .* g.val;
        v = abs(arg_p);
        mask = v > var.alpha .* var.lambda_vec;
        mask = mask .* maskS;

        % Actualización de valores
        x.val = sign(arg_p) .* (mask .* v - var.alpha .* var.lambda_vec);

        % Gradiente
        Ax = Ax_fun(x, opt, var);
        z = Ax - b;
        ATz = ATx_fun(z, opt, var);
        g = ATz;

        % Tamaño de paso BB1
        var.alpha = step_fun(x, g, opt, var, i);

        % Estadísticas
        stats.fx(i+1) = 0.5 * sum(z(:).*z(:));  % Función de Error
        stats.l1(i+1) = norm(x.val .* var.c_mask, 1); % Función de la norma l1
        stats.F(i+1) = stats.fx(i+1) + opt.lambda .* stats.l1(i+1);  % F. de Costo
        maskS = ones(length(x.val), 1);
        var.time(i+1) = toc;

    end

end

### Función Ax

In [ ]:
% Función A*x, empleada para la gradiente y las funciones de Error y Costo
function Ax = Ax_fun(z, opt, var)
    switch opt.dict
        % En el caso de datos sintéticos aleatorios
        case 'rand'
            Ax = var.A * z.val;

        % En el caso de imágenes
        case 'wave'
            Ax = waverec2(z.val, z.size, opt.wname);
    end
end

### Función ATx

In [ ]:
% Función A'*x, empleada para el cálculo de la gradiente (A'*(A*x-b))
function ATx = ATx_fun(z, opt, var)
    switch opt.dict
        % En el caso de datos sintéticos aleatorios
        case 'rand'
            ATx.val = var.A' * z;

        % En el caso de imágenes
        case 'wave'
            [ATx.val, ATx.size] = wavedec2(z, opt.n, opt.wname);
            ATx.val = ATx.val';
    end
end

### Generación de datos iniciales

In [ ]:
% Función para la generación de datos iniciales y variables necesarias
function [x_orig, b, x, var] = data_gen(opt)
    switch opt.dict
        % Caso de datos sintéticos aleatorios
        case 'rand'
            % Matriz A aleatoria normalizada
            var.A = sqrt(1/opt.N) * randn(opt.N, opt.N);
            % Datos originales (vector de datos aleatorios)
            x_orig = randn(opt.N, 1);
            % Generar datos originales que sean sparse
            ind = randperm(opt.N);
            R = 0.25 * (opt.N);
            x_orig(ind(1:R)) = 0;
            % Datos observados b
            b = var.A * x_orig + opt.sigma * randn(opt.N, 1);
            % Datos iniciales aleatorios
            x.val = randn(opt.N, 1);
            % Vector de lambdas (necesario para shrinkage y Warm Start)
            var.c_mask = ones(opt.N, 1);
            var.lambda_vec = opt.lambda * var.c_mask;

        % Caso de imágenes
        case 'wave'
            % Datos originales: lectura de imagen
            x_orig = imread(opt.sample_img);
            S = length(x_orig);
            x_orig = double(x_orig) ./ 255.0;
            % Imagen observada
            b = x_orig;
            b = b + opt.sigma * randn(S, S);
            % Datos iniciales de las mismas dimensiones
            [x.val, x.size] = wavedec2(b, opt.n, opt.wname);
            opt.N = length(x.val);
            x.val = randn(opt.N, 1);
            % Vector de lambdas necesario para shrinkage y Warm Start
            cA_size = x.size(1, 1) * x.size(1, 2);
            cD_size = sum(x.size(2:opt.n+1, 1) .* x.size(2:opt.n+1, 2)) * 3;
            var.c_mask = [zeros(cA_size, 1); ones(cD_size, 1)];
            % Lambda solo influye en coeficientes de detalle
            var.lambda_vec = opt.lambda * var.c_mask;
    end
end

### Función de tamaño de paso automático

In [ ]:
% Función de tamaño de paso automático
function alpha = step_fun(x, g, opt, var, i, alphaCi)

    % Si se encuentra en la iteración 0, se calcula el tamaño de paso de Cauchy
    if ((isequal(opt.stepT, 'BB1')) || (isequal(opt.stepT, 'YnB'))) && i == 0
        Ag = Ax_fun(g, opt, var);  % Cálculo de A*g
        alpha0 = norm(g.val)^2 / norm(Ag)^2;  % Tamaño de paso de Cauchy
        alpha0 = opt.step0_mult * alpha0;  % Multiplicación por un coeficiente

        % Límites para el tamaño de paso (opcional)
        if opt.steplim
            if alpha0 > opt.stepCH
                alpha0 = opt.stepCH;
            end
            if alpha0 < opt.stepCL
                alpha0 = opt.stepCL;
            end
        end
    end

    % Selección del tipo de tamaño de paso
    switch opt.stepT

        % Tamaño de paso constante
        case 'cte'
            alpha = opt.alpha;

        % Tamaño de paso de Barzilai-Borwein
        case 'BB1'
            if i == 0
                alpha = alpha0;  % Si es iteración 0, usar Cauchy
            else
                % Diferencia entre gradiente y valores de iteración actual y pasada
                s = x.val - var.x_ant;
                y = g.val - var.g_ant;
                alpha = s' * y / norm(y)^2;  % Tamaño de paso BB1
                alpha = opt.step_mult * alpha;  % Multiplicación por un coeficiente

                % Límites para el tamaño de paso (opcional)
                if opt.steplim
                    if alpha > opt.stepH
                        alpha = opt.stepH;
                    end
                    if alpha < opt.stepL
                        alpha = opt.stepL;
                    end
                end
            end

        % Tamaño de paso de Yuan B
        case 'YnB'
            if i == 0
                alpha = alpha0;  % Si es iteración 0, usar Cauchy
            else
                % Intercalamiento entre tamaño de paso de Cauchy y Yuan
                f = mod(i, 3);
                if i == 3 || f
                    % Tamaño de paso de Cauchy
                    alpha = alpha0;
                else
                    % Tamaño de paso de Yuan B
                    s = x.val - var.x_ant;
                    alpha = 2 / ((((1 / alphaCi) - 1 / (alpha0))^2 + (4 * norm(g.val)^2) / ...
                        (norm(s)^2))^(1/2) + 1 / alphaCi + 1 / alpha0);
                end
                alpha = opt.step_mult * alpha;  % Multiplicación por un coeficiente

                % Límites para el tamaño de paso (opcional)
                if opt.steplim
                    if alpha > opt.stepH
                        alpha = opt.stepH;
                    end
                    if alpha < opt.stepL
                        alpha = opt.stepL;
                    end
                end
            end
    end
end

### Función de Warm Start

In [ ]:
% Función de Warm Start
function lambda_vec = warms_fun(opt, var, i)

    if i > (opt.warms_iter)
        lambda_vec = var.lambda_vec_init;  % Vector inicial de lambdas
    else
        switch opt.wtype

            % Decrecimiento lineal hallado en base a dos puntos
            case 'line'
                lambda_vec = var.lambda_vec_ws * (((1 - opt.ws_inc) * i + ...
                    (opt.warms_iter * opt.ws_inc + opt.ws_inc - 1)) / ...
                    (opt.warms_iter * opt.ws_inc));

            % Decrecimiento cuadrático hallado en base al vértice
            % (derivada = 0) y otro punto
            case 'cuad'
                lambda_vec = var.lambda_vec_ws * (((1 - opt.ws_inc) * i * i + ...
                    (opt.ws_inc - 1) * 2 * i + ...
                    (opt.warms_iter * opt.warms_iter * opt.ws_inc + 1 - opt.ws_inc)) / ...
                    (opt.warms_iter * opt.warms_iter * opt.ws_inc));

            % Caso con lambda continuo, sin decrecimiento
            case 'cont'
                lambda_vec = var.lambda_vec_ws;

        end
    end
end

### Función de Screening

In [ ]:
% Función de Screening
function mask_SC = screening_fun(b, opt, var, x)
    switch opt.dict

        % Caso de datos sintéticos aleatorios
        case 'rand'
            lambda_max = max(b' * var.A);  % Calculo de lambda_max
            lambda = opt.screeningfeatures * lambda_max;  % Porcentaje de
            % lambda_max a ser evaluado
            normb = norm(b);  % norma l_2 de b
            normAi = 0.0394 * vecnorm(var.A, 1)';  % Aproximación de la norma l2
            % de las columnas de A en base a la norma l1
            absbA = abs(b' * var.A)';
            pk = (normb .* normAi + absbA) ./ (normb .* normAi + lambda_max);  % Cálculo del vector pk
            pklambdaMax = pk * lambda_max;  % Vector pk * lambda_max
            % Comparación de lambda y pk * lambda_max
            mask_SC = lambda < pklambdaMax;  % Máscara lógica de 0's y 1's
            mask_SC = double(mask_SC);  % Máscara de doubles

        % Caso de imágenes
        case 'wave'
            c = x.val;  % Valores de la transformada wavelet de la imagen
            s = x.size;  % Tamaño de los niveles de la transformada
            c_size = length(c);  % Tamaño de todos los coeficientes
            cA_size = s(1, 1) * s(1, 2);  % Tamaño de los coeficientes aproximados
            cD_size = c_size - cA_size;  % Tamaño de los coeficientes de detalle
            c_mask = [zeros(cA_size, 1); ones(cD_size, 1)];  % Máscara de 0's
            % (coeficientes aproximados) y 1's (coeficientes de detalle)
            c = c_mask .* c;  % Se evalúan sólo los coeficientes de detalle
            lambda_max = max(c);  % Máximo valor de los coeficientes de detalle

            lambda = opt.screeningfeatures * lambda_max;  % Porcentaje de
            % lambda_max a ser evaluado
            xk2 = (norm(c) / norm(b)) / length(c);  % Aproximación del cálculo de
            % la norma de las columnas de A
            y2 = norm(b);
            pk = (y2 * xk2 + abs(c)) / (y2 * xk2 + lambda_max);  % Cálculo del vector pk
            pklambdaMax = pk * lambda_max;  % Vector pk * lambda_max
            % Comparación de lambda y pk * lambda_max
            mask_SC = lambda < pklambdaMax;  % Máscara lógica de 0's y 1's
            mask_SC = double(mask_SC);  % Máscara de doubles
            mask_SC = mask_SC .* c_mask;  % Máscara que sólo toma los
            % coeficientes de detalle)
            mask_SC(1:cA_size) = ones(cA_size, 1);  % Máscara de coeficientes
            % aproximados sin cambiar (1's)

    end
end

### Bloque de inicialización y configuración

In [ ]:
clc;
clear all;
close all;

% Opciones de selección
opt.dict = 'rand';       % Diccionario: 'rand' o 'wave'
opt.stepT = 'cte';       % Nombre del tamaño de paso: 'cte', 'BB1', 'YnB'
opt.wtype = 'cont';      % Estrategia de Warm Start: 'cont', 'line', 'cuad'
opt.wname = 'db4';       % Tipo de transformada Wavelet: 'db1', 'db4', 'bior4.4'

% Constantes lógicas
opt.steplim = 0;         % Activación de límites para el tamaño de paso
opt.warms = 0;           % Activación de Warm Start
opt.screening = 0;       % Activación de Screening

% Parámetros
opt.N = 1000;            % Tamaño de los valores a evaluar
L = 50;                  % Número de iteraciones
opt.alpha = 0.1;         % Tamaño de paso
opt.lambda = 0.1;        % Parámetro de regularización
opt.sigma = 1.0 * 10^-2; % Ruido aditivo
opt.ws_inc = 2;          % Incremento para Warm Start
opt.warms_iter = 0.1 * L;% Iteración hasta la cual se evalúa Warm Start
opt.screeningfeatures = 1; % Intensidad de Screening
opt.n = 3;               % Nivel de evaluación de la transformada Wavelets

### Generación de datos y variables iniciales

In [ ]:
% Generación de datos y variables iniciales
[x_orig, b, x_init, var_init] = data_gen(opt);

### Caso: ISTA

In [ ]:
% ISTA
x = x_init;
var = var_init;
[stats_ista, x_ista, var_ista] = general_ista(opt, var, b, x, L);

### Caso: FISTA

In [ ]:
% FISTA
x = x_init;
var = var_init;
[stats_fista, x_fista, var_fista] = general_fista(opt, var, b, x, L);

### Caso: FISTA TPAL (Tamaño de paso automático con límites)

In [ ]:
% FISTA TPAL
opt.stepT = 'BB1';
opt.steplim = 1;
opt.stepCH = 0.2;     % Valor máximo del tamaño de paso de Cauchy
opt.stepCL = 0.1;     % Valor mínimo del tamaño de paso de Cauchy
opt.stepH = 0.4;      % Valor máximo del tamaño de paso automático
opt.stepL = 0.13;     % Valor mínimo del tamaño de paso automático
opt.step0_mult = 0.35; % Coeficiente para el tamaño de paso de Cauchy
opt.step_mult = 0.35;  % Coeficiente para el tamaño de paso automático

x = x_init;
var = var_init;
[stats_fista_auto, x_fista_auto, var_fista_auto] = general_fista(opt, var, b, x, L);

### Caso: FISTA WS (Warm Start) suave

In [ ]:
% FISTA WS suave
opt.warms = 1;
opt.stepT = 'cte';
opt.ws_inc = 2;

x = x_init;
var = var_init;
[stats_fista_wsL, x_fista_wsL, var_fista_wsL] = general_fista(opt, var, b, x, L);

### Caso: FISTA WS fuerte

In [ ]:
% FISTA WS fuerte
opt.ws_inc = 5;

x = x_init;
var = var_init;
[stats_fista_wsM, x_fista_wsM, var_fista_wsM] = general_fista(opt, var, b, x, L);

### Caso: FISTA WS excesivo

In [ ]:
% FISTA WS excesivo
opt.ws_inc = 10;

x = x_init;
var = var_init;
[stats_fista_wsH, x_fista_wsH, var_fista_wsH] = general_fista(opt, var, b, x, L);

### Caso: FISTA SC (Screening) suave

In [ ]:
% FISTA SC suave
opt.warms = 0;
opt.screening = 1;
opt.screeningfeatures = 0.89;

x = x_init;
var = var_init;
[stats_fista_scL, x_fista_sL, var_fista_scL] = general_fista(opt, var, b, x, L);

### Caso: FISTA SC medio

In [ ]:
% FISTA SC medio
opt.screeningfeatures = 0.91;

x = x_init;
var = var_init;
[stats_fista_scM, x_fista_scM, var_fista_scM] = general_fista(opt, var, b, x, L);

### Caso: FISTA SC fuerte

In [ ]:
% FISTA SC fuerte
opt.screeningfeatures = 1;

x = x_init;
var = var_init;
[stats_fista_scH, x_fista_scH, var_fista_scH] = general_fista(opt, var, b, x, L);

### Caso: FISTA OPT (combinación óptima)

In [ ]:
% FISTA OPT
opt.warms = 1;
opt.ws_inc = 2;
opt.screening = 1;
opt.screeningfeatures = 1;
opt.stepT = 'BB1';
opt.steplim = 1;
opt.step0_mult = 0.35; % Coeficiente para Cauchy
opt.step_mult = 0.35;  % Coeficiente para automático
opt.stepCH = 0.2;      % Máximo Cauchy
opt.stepCL = 0.1;      % Mínimo Cauchy
opt.stepH = 0.25;      % Máximo automático
opt.stepL = 0.12;      % Mínimo automático

x = x_init;
var = var_init;
[stats_fista_all, x_fista_all, var_fista_all] = general_fista(opt, var, b, x, L);

### Gráficos de comparación de algoritmos

In [ ]:
% Comparación entre FISTA Regular, TPAL, WSO, SCO, y OPT
figure(1);
subplot(1,3,1), semilogy(stats_fista.fx, 'LineWidth', 3), hold on;
semilogy(stats_fista_auto.fx, 'LineWidth', 3);
semilogy(stats_fista_wsM.fx, 'LineWidth', 3);
semilogy(stats_fista_scH.fx, 'LineWidth', 3);
semilogy(stats_fista_all.fx, 'LineWidth', 3);
set(gca, 'FontSize', 24);
grid;
xlabel('Iteraciones'), ylabel('Funcion de error');
xlim([1 50]);

subplot(1,3,2), semilogy(stats_fista.l1, 'LineWidth', 3), hold on;
semilogy(stats_fista_auto.l1, 'LineWidth', 3);
semilogy(stats_fista_wsM.l1, 'LineWidth', 3);
semilogy(stats_fista_scH.l1, 'LineWidth', 3);
semilogy(stats_fista_all.l1, 'LineWidth', 3);
set(gca, 'FontSize', 24);
grid;
xlabel('Iteraciones'), ylabel('Norma l1 de x');
xlim([1 50]);

subplot(1,3,3), semilogy(stats_fista.F, 'LineWidth', 3), hold on;
semilogy(stats_fista_auto.F, 'LineWidth', 3);
semilogy(stats_fista_wsM.F, 'LineWidth', 3);
semilogy(stats_fista_scH.F, 'LineWidth', 3);
semilogy(stats_fista_all.F, 'LineWidth', 3);
set(gca, 'FontSize', 24);
legend('FISTA', 'FISTA TPAL', 'FISTA WSO', 'FISTA SCO', 'FISTA OPT', 'FontSize', 26);
grid;
xlabel('Iteraciones'), ylabel('Funcion de costo F(x)');
xlim([1 50]);

### Gráficos de comparación por tiempo

In [ ]:
% Comparación de tiempo para FISTA Regular, TPAL, WSO, SCO, y OPT
figure(2);
semilogy(var_fista.time(1:L), stats_fista.F(1:L), 'LineWidth', 3), hold on;
semilogy(var_fista_auto.time(1:L), stats_fista_auto.F(1:L), 'LineWidth', 3);
semilogy(var_fista_wsM.time(1:L), stats_fista_wsM.F(1:L), 'LineWidth', 3);
semilogy(var_fista_scH.time(1:L), stats_fista_scH.F(1:L), 'LineWidth', 3);
semilogy(var_fista_all.time(1:L), stats_fista_all.F(1:L), 'LineWidth', 3);
set(gca, 'FontSize', 24);
legend('FISTA', 'FISTA TPAL', 'FISTA WSO', 'FISTA SCO', 'FISTA OPT', 'FontSize', 26);
grid;
xlabel('Tiempo (s)'), ylabel('Funcion de Costo F(x)');
xlim([0 0.11]);

### Bloque de inicialización y configuración

In [ ]:
clc;
clear all;
close all;

% Opciones de selección
opt.dict = 'wave';           % Diccionario: 'rand' o 'wave'
opt.stepT = 'cte';           % Nombre del tamaño de paso: 'cte', 'BB1', 'YnB'
opt.wtype = 'cont';          % Estrategia de Warm Start: 'cont', 'line', 'cuad'
opt.wname = 'db4';           % Tipo de Transformada Wavelet: 'db1', 'db4', 'bior4.4'
opt.sample_img = 'baboon.gif'; % Imagen de entrada: 'lena.bmp', 'barbara.bmp', 'goldhill.gif', etc.

% Constantes lógicas para activación de opciones
opt.steplim = 0;  % Activación de límites de tamaño de paso
opt.warms = 0;    % Activación de Warm Start
opt.screening = 0; % Activación de Screening

% Parámetros
L = 20;                     % Número de iteraciones
opt.alpha = 0.05;           % Tamaño de paso
opt.lambda = 80 * 10^-3;    % Parámetro de regularización
opt.sigma = 1.5 * 10^-1;    % Ruido aditivo
opt.ws_inc = 2;             % Incremento para Warm Start
opt.warms_iter = 5;         % Iteración hasta la cual se evalúa Warm Start
opt.screeningfeatures = 1;  % Intensidad de Screening
opt.n = 3;                  % Nivel de evaluación de la Transformada Wavelets

### Generación de datos y variables iniciales

In [ ]:
% Generación de datos y variables iniciales
[x_orig, b, x_init, var_init] = data_gen(opt);

### Caso: ISTA

In [ ]:
% ISTA
x = x_init;
var = var_init;
[stats_ista, x_ista, var_ista] = general_ista(opt, var, b, x, L);

### Caso: FISTA

In [ ]:
% FISTA
x = x_init;
var = var_init;
[stats_fista, x_fista, var_fista] = general_fista(opt, var, b, x, L);

### Caso: FISTA TPAL (tamaño de paso automático con límites)

In [ ]:
% FISTA TPAL
opt.stepT = 'BB1';
opt.steplim = 1;
opt.stepCH = 0.2;        % Máximo tamaño de paso de Cauchy
opt.stepCL = 0.1;        % Mínimo tamaño de paso de Cauchy
opt.stepH = 0.3;         % Máximo tamaño de paso automático
opt.stepL = 0.2;         % Mínimo tamaño de paso automático
opt.step0_mult = 0.1;    % Coeficiente para tamaño de paso de Cauchy
opt.step_mult = 0.3;     % Coeficiente para tamaño de paso automático

x = x_init;
var = var_init;
[stats_fista_auto, x_fista_auto, var_fista_auto] = general_fista(opt, var, b, x, L);

### Caso: FISTA WS (Warm Start) suave

In [ ]:
% FISTA WS suave
opt.warms = 1;
opt.stepT = 'cte';
opt.ws_inc = 2;

x = x_init;
var = var_init;
[stats_fista_wsL, x_fista_wsL, var_fista_wsL] = general_fista(opt, var, b, x, L);

### Caso: FISTA WS fuerte

In [ ]:
% FISTA WS fuerte
opt.ws_inc = 10;

x = x_init;
var = var_init;
[stats_fista_wsM, x_fista_wsM, var_fista_wsM] = general_fista(opt, var, b, x, L);

### Caso: FISTA WS excesivo

In [ ]:
% FISTA WS excesivo
opt.ws_inc = 20;

x = x_init;
var = var_init;
[stats_fista_wsH, x_fista_wsH, var_fista_wsH] = general_fista(opt, var, b, x, L);

### Caso: FISTA SC (Screening) suave

In [ ]:
% FISTA SC suave
opt.warms = 0;
opt.screening = 1;
opt.screeningfeatures = 0.3;

x = x_init;
var = var_init;
[stats_fista_scL, x_fista_sL, var_fista_scL] = general_fista(opt, var, b, x, L);

### Caso: FISTA SC medio

In [ ]:
% FISTA SC medio
opt.screeningfeatures = 0.5;

x = x_init;
var = var_init;
[stats_fista_scM, x_fista_scM, var_fista_scM] = general_fista(opt, var, b, x, L);

### Caso: FISTA SC fuerte

In [ ]:
% FISTA SC fuerte
opt.screeningfeatures = 1;

x = x_init;
var = var_init;
[stats_fista_scH, x_fista_scH, var_fista_scH] = general_fista(opt, var, b, x, L);

### Caso: FISTA OPT (combinación óptima de opciones)

In [ ]:
% FISTA OPT
opt.warms = 1;
opt.ws_inc = 1.5;
opt.screening = 1;
opt.screeningfeatures = 1;
opt.stepT = 'BB1';
opt.steplim = 1;
opt.step0_mult = 0.1; % Coeficiente para Cauchy
opt.step_mult = 0.3;  % Coeficiente para automático
opt.stepCH = 0.2;     % Máximo Cauchy
opt.stepCL = 0.1;     % Mínimo Cauchy
opt.stepH = 0.3;      % Máximo automático
opt.stepL = 0.2;      % Mínimo automático

x = x_init;
var = var_init;
[stats_fista_all, x_fista_all, var_fista_all] = general_fista(opt, var, b, x, L);

### Gráficos de comparación de algoritmos

In [ ]:
% Comparación entre FISTA Regular, TPAL, WSO, SCO y OPT
figure(1);

subplot(1,3,1), plot(stats_fista.fx, 'LineWidth', 3), hold on;
plot(stats_fista_auto.fx, 'LineWidth', 3);
plot(stats_fista_wsM.fx, 'LineWidth', 3);
plot(stats_fista_scH.fx, 'LineWidth', 3);
plot(stats_fista_all.fx, 'LineWidth', 3);
set(gca, 'FontSize', 24);
grid;
xlabel('Iteraciones'), ylabel('Funcion de error');
xlim([1 20]);

subplot(1,3,2), plot(stats_fista.l1, 'LineWidth', 3), hold on;
plot(stats_fista_auto.l1, 'LineWidth', 3);
plot(stats_fista_wsM.l1, 'LineWidth', 3);
plot(stats_fista_scH.l1, 'LineWidth', 3);
plot(stats_fista_all.l1, 'LineWidth', 3);
set(gca, 'FontSize', 24);
grid;
xlabel('Iteraciones'), ylabel('Norma l1 de x');
xlim([1 20]);

subplot(1,3,3), plot(stats_fista.F, 'LineWidth', 3), hold on;
plot(stats_fista_auto.F, 'LineWidth', 3);
plot(stats_fista_wsM.F, 'LineWidth', 3);
plot(stats_fista_scH.F, 'LineWidth', 3);
plot(stats_fista_all.F, 'LineWidth', 3);
set(gca, 'FontSize', 24);
legend('FISTA', 'FISTA TPAL', 'FISTA WSO', 'FISTA SCO', 'FISTA OPT', 'FontSize', 26);
grid;
xlabel('Iteraciones'), ylabel('Funcion de costo F(x)');
xlim([1 20]);

### Gráficos de comparación por tiempo

In [ ]:
% Comparación del tiempo para FISTA Regular, TPAL, WSO, SCO y OPT
figure(2);

plot(var_fista.time(1:L), stats_fista.F(1:L), 'LineWidth', 3), hold on;
plot(var_fista_auto.time(1:L), stats_fista_auto.F(1:L), 'LineWidth', 3);
plot(var_fista_wsM.time(1:L), stats_fista_wsM.F(1:L), 'LineWidth', 3);
plot(var_fista_scH.time(1:L), stats_fista_scH.F(1:L), 'LineWidth', 3);
plot(var_fista_all.time(1:L), stats_fista_all.F(1:L), 'LineWidth', 3);
set(gca, 'FontSize', 24);
legend('FISTA', 'FISTA TPAL', 'FISTA WSO', 'FISTA SCO', 'FISTA OPT', 'FontSize', 26);
grid;
xlabel('Tiempo (s)'), ylabel('Funcion de Costo F(x)');
xlim([0 2.7]);